<a href="https://colab.research.google.com/github/alessandropessoa/LLM-cases/blob/main/formas_de_treinamento_llms_fine_tune_extracao_feature_lora_destilacao_conheimento_testes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DESTILAÇÂO DE CONHECIMENTO**

#Visão geral

**Destilação de conhecimento**

**O que?**

É uma técnica usada para transferir o conhecimento de um modelo grande e poderoso (chamado de professor ou teacher) para um modelo menor e mais leve (chamado de aluno ou student).

**Por que?**

* Modelos grandes (como BERT, GPT, etc.) são muito precisos, mas pesados e lentos para usar em produção (celulares, IoT, etc.).

* Modelos pequenos são mais rápidos, leves, mas geralmente menos precisos.

* A destilação de conhecimento busca ensinar o modelo pequeno a imitar o comportamento do modelo grande, alcançando boa performance com menos recursos


**Como ?**

* O modelo professor é treinado com os dados e gera "soft labels":

        Ao invés de apenas dizer “a resposta é A”, ele dá uma distribuição de probabilidade, tipo:

            A: 0.82

            B: 0.13

            C: 0.04

            D: 0.01

* O modelo aluno aprende com essas soft labels, que são mais informativas do que as etiquetas tradicionais (0 ou 1).

        Isso ajuda o aluno a aprender nuances e relações sutis entre as classes.

* A função de perda (loss) é uma combinação de duas partes:

        Uma compara a saída do aluno com os rótulos reais.

        Outra compara com a saída do professor (soft labels), geralmente usando uma função de entropia cruzada suavizada por temperatura (softmax com temperatura).

fORUMLA:

$$
\text{Loss} = \alpha \cdot \text{CE}(y_{\text{student}}, y_{\text{true}}) + (1 - \alpha) \cdot \text{KL}(y_{\text{student}}^{(T)} \parallel y_{\text{teacher}}^{(T)})
$$

Explicação dos termos:

* Loss: Função de perda total usada para treinar o modelo student (aluno).

* α: Peso (entre 0 e 1) que controla a importância entre os dois tipos de perda.

* CE: Cross-Entropy Loss — mede a diferença entre a saída do aluno e os rótulos reais (y_true).

* KL: Kullback-Leibler Divergence — mede a diferença entre a distribuição de probabilidade do aluno e a do professor, ambas suavizadas por uma temperatura T.

* ystudent(T)ystudent(T)​: Saída do aluno com softmax ajustado por temperatura T (mais suave).

* yteacher(T)yteacher(T)​: Saída do professor com softmax T.

**A temperatura no softmax é usada para “suavizar” a distribuição de probabilidades. Fórmula do softmax com temperatura:**
$$
\text{softmax}_T(z_i) = \frac{e^{z_i / T}}{\sum_j e^{z_j / T}}
$$


* Quando T > 1, o softmax gera probabilidades mais suaves (menos confiantes), revelando relações sutis entre classes.

* Isso ajuda o modelo aluno a entender melhor como o professor vê os dados, mesmo quando erra por pouco.

 Intuição:

* O Cross-Entropy com os rótulos reais garante que o aluno aprenda a tarefa corretamente.

* A KL Divergence com o professor ajuda o aluno a “copiar” o comportamento inteligente do professor, incluindo conhecimento implícito entre classes.

# Pipeline de trabalho e objetivo

### Objetivo
  Destilar conhecimeneto de modelos maior para menor, ou seja do professor (modelo grande) para o modelo aluno (menor menor)

# importação de depedências

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import namedtuple

#Simulação

Essa classee é modelo maior, tipo o BERT LARGE

In [12]:
class   ModelpProfessor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 10)  # 10 classes
        )

    def forward(self, x):
        return self.fc(x)

Essa classer é modelo menor, tipo o BERT BASE

In [13]:
class ModeloEstudante(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(100, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.fc(x)

In [14]:
professor = ModelpProfessor()
estudante= ModeloEstudante()

No exemplo abaixo iremos simula um batch com 32 amostras e 100 featrures, e são 10 tipos de rotulos que se pode ter para as 32 amostras

In [9]:
# Dados fictícios
x = torch.randn(32, 100)            # batch com 32 amostras, 100 features
y_true = torch.randint(0, 10, (32,))  # rótulos reais

In [10]:
x

tensor([[-0.6587, -1.7000, -0.1837,  ...,  1.8617, -1.5898,  0.8192],
        [ 0.5071, -1.1964,  0.1379,  ...,  0.2671, -0.5135,  1.7725],
        [-0.3025,  0.3318,  1.2593,  ...,  1.3519, -1.3106,  0.6065],
        ...,
        [-0.9346, -0.5077, -1.8255,  ..., -0.0740,  0.0888,  0.3412],
        [ 1.7038, -0.0305, -0.3958,  ..., -1.1218, -0.6705,  0.2440],
        [ 0.4347, -1.2985, -0.0836,  ..., -0.6392,  2.6205,  1.3970]])

In [15]:
x.shape

torch.Size([32, 100])

In [38]:
y_true

tensor([8, 3, 2, 3, 3, 0, 7, 3, 1, 6, 9, 3, 9, 3, 9, 9, 3, 1, 1, 8, 7, 0, 7, 1,
        7, 3, 0, 9, 7, 8, 2, 3])

In [37]:
y_true.shape

torch.Size([32])

**A temperatura no softmax é usada para “suavizar” a distribuição de probabilidades. Fórmula do softmax com temperatura:**
$$
\text{softmax}_T(z_i) = \frac{e^{z_i / T}}{\sum_j e^{z_j / T}}
$$


* Quando T > 1, o softmax gera probabilidades mais suaves (menos confiantes), revelando relações sutis entre classes.

* Isso ajuda o modelo aluno a entender melhor como o professor vê os dados, mesmo quando erra por pouco.

 Intuição:

* O Cross-Entropy com os rótulos reais garante que o aluno aprenda a tarefa corretamente.

* A KL Divergence com o professor ajuda o aluno a “copiar” o comportamento inteligente do professor, incluindo conhecimento implícito entre classes.

T é a temperatura: quanto maior, mais suave (mais distribuída) fica a saída.

Quando T=1, a fórmula vira o softmax comum.

Quando T>1, a distribuição das probabilidades fica mais "suave", revelando melhor as relações entre as classes — ideal para destilação de conhecimento.

Quando T→0, o softmax se aproxima de uma distribuição one-hot.

In [16]:
# Temperatura e alpha
T = 2.0
alpha = 0.7

In [23]:
# Saídas dos modelos
with torch.no_grad():  # o professor já foi treinado
    professor_logits = professor(x)
    professor_probs = F.softmax(professor_logits / T, dim=1)

In [24]:
professor_logits

tensor([[-0.1278, -0.2230, -0.0483,  0.0105, -0.0974, -0.1367, -0.1930, -0.0457,
          0.4329,  0.4156],
        [-0.1638, -0.1079,  0.0765, -0.1543, -0.1790,  0.0633,  0.6179, -0.0825,
         -0.0664, -0.0310],
        [-0.4422,  0.1340,  0.0926,  0.2089, -0.2680, -0.0099, -0.0025, -0.0585,
         -0.0823,  0.3825],
        [-0.2592,  0.4054,  0.2289,  0.2923, -0.3748,  0.2991,  0.2390,  0.2221,
          0.1086,  0.2760],
        [-0.1758,  0.0212,  0.2482, -0.2482,  0.2731, -0.1753,  0.0401, -0.0872,
         -0.3104,  0.2291],
        [-0.0009,  0.2136,  0.0623,  0.1910, -0.0715,  0.0110,  0.1072,  0.0479,
         -0.2553,  0.1060],
        [-0.3049,  0.2214,  0.2453,  0.1484, -0.1260, -0.0477,  0.0910, -0.2332,
          0.0898, -0.1025],
        [ 0.0886,  0.0861,  0.0587, -0.1767, -0.0373, -0.0242,  0.0301, -0.2036,
          0.0562,  0.1572],
        [-0.0876,  0.0878, -0.2286,  0.2896,  0.0310, -0.2253,  0.1596, -0.3043,
         -0.0921,  0.1685],
        [-0.3283, -

In [22]:
professor_logits.shape

torch.Size([32, 10])

In [25]:
professor_probs

tensor([[0.0933, 0.0889, 0.0970, 0.0999, 0.0947, 0.0929, 0.0903, 0.0972, 0.1234,
         0.1224],
        [0.0916, 0.0942, 0.1033, 0.0921, 0.0910, 0.1027, 0.1355, 0.0954, 0.0962,
         0.0979],
        [0.0799, 0.1065, 0.1043, 0.1106, 0.0871, 0.0991, 0.0995, 0.0967, 0.0956,
         0.1206],
        [0.0812, 0.1132, 0.1036, 0.1070, 0.0766, 0.1073, 0.1041, 0.1033, 0.0976,
         0.1061],
        [0.0920, 0.1015, 0.1137, 0.0887, 0.1151, 0.0920, 0.1024, 0.0961, 0.0860,
         0.1126],
        [0.0977, 0.1088, 0.1009, 0.1076, 0.0943, 0.0983, 0.1032, 0.1001, 0.0861,
         0.1031],
        [0.0856, 0.1114, 0.1127, 0.1074, 0.0936, 0.0973, 0.1043, 0.0887, 0.1043,
         0.0947],
        [0.1042, 0.1041, 0.1026, 0.0912, 0.0978, 0.0985, 0.1012, 0.0900, 0.1025,
         0.1078],
        [0.0963, 0.1051, 0.0897, 0.1162, 0.1021, 0.0899, 0.1089, 0.0864, 0.0960,
         0.1094],
        [0.0901, 0.0974, 0.1180, 0.0883, 0.0899, 0.0961, 0.1090, 0.1045, 0.0926,
         0.1141],
        [0

In [26]:
professor_probs.shape

torch.Size([32, 10])

In [ ]:
estudante_logits = estudante(x)
estudante_probs = F.log_softmax(estudante_logits / T, dim=1)

**Perdas**

 # com rótulos reais

In [28]:
ce_loss = F.cross_entropy(student_logits, y_true)  # com rótulos reais
ce_loss

tensor(2.3908, grad_fn=<NllLossBackward0>)

# com destilação

In [31]:
kl_loss = F.kl_div(student_probs, teacher_probs, reduction="batchmean") * (T * T)  # com distilação
kl_loss

tensor(0.0554, grad_fn=<MulBackward0>)

# Perda final combinada

In [34]:
# Loss final combinada
loss = alpha * ce_loss + (1 - alpha) * kl_loss
loss

tensor(1.6902, grad_fn=<AddBackward0>)

In [35]:
print(f"PerdA total: {loss.item():.4f}")


PerdA total: 1.6902


Resumo:

O professor foi treinado antes (aqui estamos simulando).

O estudante aprende tanto com os rótulos reais quanto com a sabedoria do Teacher.

A distilação acontece com softmax suavizado, revelando relações entre classes que o rótulo direto não mostra.


Multiplicamos o kl_loss por T² por recomendação do artigo original de Hinton (Knowledge Distillation, 2015).

#TESTE 2

In [200]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import namedtuple
import pandas as pd

In [202]:
ds = pd.read_csv("/content/textos_categorizados.csv")


In [219]:
ds

,texto,label
0,Organizações civis e movimentos sociais convoc...,protesto
1,Milhares de manifestantes ocuparam as ruas da ...,protesto
2,Protestos se intensificaram após o aumento no ...,protesto
3,Integrantes de diversos sindicatos realizaram ...,protesto
4,Estudantes de várias universidades públicas or...,protesto
5,O relatório do Banco Central apontou crescimen...,economia
6,A inflação do mês de março apresentou uma leve...,economia
7,Analistas financeiros revisaram as projeções d...,economia
8,A taxa básica de juros foi reduzida pelo Comit...,economia
9,A bolsa de valores fechou o pregão com alta de...,economia


In [220]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   texto   20 non-null     object
 1   label   20 non-null     object
dtypes: object(2)
memory usage: 452.0+ bytes


In [221]:
ds.label.unique()

array(['protesto', 'economia', 'educacao', 'meio_ambiente'], dtype=object)

In [222]:
ds = ds.drop_duplicates().reset_index(drop=True)
ds

,texto,label
0,Organizações civis e movimentos sociais convoc...,protesto
1,Milhares de manifestantes ocuparam as ruas da ...,protesto
2,Protestos se intensificaram após o aumento no ...,protesto
3,Integrantes de diversos sindicatos realizaram ...,protesto
4,Estudantes de várias universidades públicas or...,protesto
5,O relatório do Banco Central apontou crescimen...,economia
6,A inflação do mês de março apresentou uma leve...,economia
7,Analistas financeiros revisaram as projeções d...,economia
8,A taxa básica de juros foi reduzida pelo Comit...,economia
9,A bolsa de valores fechou o pregão com alta de...,economia


In [224]:
label_enc = LabelEncoder()
labels_encoded = label_enc.fit_transform(ds.label)

In [225]:
labels_encoded

array([3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2])

In [227]:
x_train, X_test, y_train, y_test = train_test_split(ds.texto, labels_encoded, test_size=0.25, random_state=42)

In [228]:
def tokenizacao(texto):
    return texto.lower().split()

In [229]:
vocab = {"<pad>":0}
for texto in x_train:
  for token in tokenizacao(texto):
    if token not in vocab:
      vocab[token] = len(vocab)
vocab

{'<pad>': 0,
 'o': 1,
 'relatório': 2,
 'do': 3,
 'banco': 4,
 'central': 5,
 'apontou': 6,
 'crescimento': 7,
 'econômico': 8,
 'moderado': 9,
 'no': 10,
 'segundo': 11,
 'trimestre,': 12,
 'com': 13,
 'destaque': 14,
 'para': 15,
 'setor': 16,
 'de': 17,
 'serviços': 18,
 'e': 19,
 'construção': 20,
 'civil.': 21,
 'uma': 22,
 'nova': 23,
 'lei': 24,
 'aprovada': 25,
 'congresso': 26,
 'estabelece': 27,
 'que': 28,
 'escolas': 29,
 'públicas': 30,
 'receberão': 31,
 'recursos': 32,
 'adicionais': 33,
 'promover': 34,
 'inclusão': 35,
 'alunos': 36,
 'deficiência': 37,
 'necessidades': 38,
 'especiais.': 39,
 'integrantes': 40,
 'diversos': 41,
 'sindicatos': 42,
 'realizaram': 43,
 'grande': 44,
 'paralisação': 45,
 'em': 46,
 'frente': 47,
 'ao': 48,
 'palácio': 49,
 'governo,': 50,
 'exigindo': 51,
 'melhores': 52,
 'condições': 53,
 'trabalho': 54,
 'aumento': 55,
 'salarial': 56,
 'imediato.': 57,
 'a': 58,
 'conferência': 59,
 'internacional': 60,
 'sobre': 61,
 'mudanças': 62,


In [230]:
for texto in X_test:
    for token in tokenizacao(texto):
        if token not in vocab:
            vocab[token] = len(vocab)

In [231]:
vocab

{'<pad>': 0,
 'o': 1,
 'relatório': 2,
 'do': 3,
 'banco': 4,
 'central': 5,
 'apontou': 6,
 'crescimento': 7,
 'econômico': 8,
 'moderado': 9,
 'no': 10,
 'segundo': 11,
 'trimestre,': 12,
 'com': 13,
 'destaque': 14,
 'para': 15,
 'setor': 16,
 'de': 17,
 'serviços': 18,
 'e': 19,
 'construção': 20,
 'civil.': 21,
 'uma': 22,
 'nova': 23,
 'lei': 24,
 'aprovada': 25,
 'congresso': 26,
 'estabelece': 27,
 'que': 28,
 'escolas': 29,
 'públicas': 30,
 'receberão': 31,
 'recursos': 32,
 'adicionais': 33,
 'promover': 34,
 'inclusão': 35,
 'alunos': 36,
 'deficiência': 37,
 'necessidades': 38,
 'especiais.': 39,
 'integrantes': 40,
 'diversos': 41,
 'sindicatos': 42,
 'realizaram': 43,
 'grande': 44,
 'paralisação': 45,
 'em': 46,
 'frente': 47,
 'ao': 48,
 'palácio': 49,
 'governo,': 50,
 'exigindo': 51,
 'melhores': 52,
 'condições': 53,
 'trabalho': 54,
 'aumento': 55,
 'salarial': 56,
 'imediato.': 57,
 'a': 58,
 'conferência': 59,
 'internacional': 60,
 'sobre': 61,
 'mudanças': 62,


In [233]:
def encode(texto):
    tokens = tokenizacao(texto)
    return [vocab[t] for t in tokens]

In [235]:
max_len =max(len(tokenizacao(t)) for t in ds.texto)
max_len

26

In [236]:
def pad_sequence(seq, max_len):
    return seq + [0] * (max_len - len(seq))

In [237]:
class NovoDataset(Dataset):
    def __init__(self, textos, labels):
        self.x = [pad_sequence(encode(t), max_len) for t in textos]
        self.y = labels

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]) ,torch.tensor(self.y[idx])

In [238]:
train_ds = NovoDataset(x_train, y_train)
test_ds = NovoDataset(X_test, y_test)

In [239]:
train_ds.x

[[1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  1,
  16,
  17,
  18,
  19,
  20,
  21,
  0,
  0,
  0,
  0],
 [22,
  23,
  24,
  25,
  10,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  15,
  34,
  35,
  17,
  36,
  13,
  37,
  19,
  38,
  39,
  0,
  0,
  0],
 [40,
  17,
  41,
  42,
  43,
  22,
  44,
  45,
  46,
  47,
  48,
  49,
  3,
  50,
  51,
  52,
  53,
  17,
  54,
  19,
  55,
  56,
  57,
  0,
  0,
  0],
 [58,
  59,
  60,
  61,
  62,
  63,
  64,
  58,
  65,
  66,
  67,
  68,
  19,
  66,
  69,
  70,
  71,
  72,
  15,
  73,
  1,
  74,
  75,
  0,
  0,
  0],
 [76,
  77,
  78,
  79,
  80,
  17,
  81,
  46,
  82,
  83,
  66,
  84,
  85,
  86,
  87,
  88,
  19,
  89,
  17,
  90,
  91,
  0,
  0,
  0,
  0,
  0],
 [1,
  92,
  66,
  93,
  94,
  79,
  95,
  17,
  96,
  15,
  97,
  66,
  98,
  99,
  13,
  100,
  101,
  102,
  103,
  104,
  35,
  105,
  19,
  106,
  107,
  0],
 [108,
  109,
  110,
  111,
  1,
  55,
  10,
  112,
  113,
  114,
  115,
  116,
  

In [240]:
train_ds.y

array([0, 1, 3, 2, 2, 1, 3, 0, 2, 3, 1, 0, 1, 1, 0])

In [241]:
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=2, shuffle=False)

In [249]:
class Classificador(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * max_len, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        emb = self.embedding(x)
        emb = emb.type(torch.float32)
        emb = emb.view(x.size(0), -1) #flatten do vetor
        emb = F.relu(self.fc1(emb))
        emb = self.fc2(emb)
        return emb

In [250]:
professor = Classificador(vocab_size=len(vocab), embedding_dim=16, hidden_dim=32, num_classes=4)

In [251]:
aluno = Classificador(vocab_size=len(vocab), embedding_dim=8, hidden_dim=16, num_classes=4)

## Função de destilação

* A função de perda (loss) é uma combinação de duas partes:

        Uma compara a saída do aluno com os rótulos reais.

        Outra compara com a saída do professor (soft labels), geralmente usando uma função de entropia cruzada suavizada por temperatura (softmax com temperatura).

fORUMLA:

$$
\text{Loss} = \alpha \cdot \text{CE}(y_{\text{student}}, y_{\text{true}}) + (1 - \alpha) \cdot \text{KL}(y_{\text{student}}^{(T)} \parallel y_{\text{teacher}}^{(T)})
$$

Explicação dos termos:

* Loss: Função de perda total usada para treinar o modelo student (aluno).

* α: Peso (entre 0 e 1) que controla a importância entre os dois tipos de perda.

* CE: Cross-Entropy Loss — mede a diferença entre a saída do aluno e os rótulos reais (y_true).

* KL: Kullback-Leibler Divergence — mede a diferença entre a distribuição de probabilidade do aluno e a do professor, ambas suavizadas por uma temperatura T.

* ystudent(T)ystudent(T)​: Saída do aluno com softmax ajustado por temperatura T (mais suave).

* yteacher(T)yteacher(T)​: Saída do professor com softmax T.

**A temperatura no softmax é usada para “suavizar” a distribuição de probabilidades. Fórmula do softmax com temperatura:**
$$
\text{softmax}_T(z_i) = \frac{e^{z_i / T}}{\sum_j e^{z_j / T}}
$$


* Quando T > 1, o softmax gera probabilidades mais suaves (menos confiantes), revelando relações sutis entre classes.

* Isso ajuda o modelo aluno a entender melhor como o professor vê os dados, mesmo quando erra por pouco.

 Intuição:

* O Cross-Entropy com os rótulos reais garante que o aluno aprenda a tarefa corretamente.

* A KL Divergence com o professor ajuda o aluno a “copiar” o comportamento inteligente do professor, incluindo conhecimento implícito entre classes.

In [252]:
def destilacao_loss(estudante_logits, professor_logits,targets, T=2.0, alpha=0.7):
    professor_probs = F.softmax(professor_logits / T, dim=1)

    estudante_log_probs = F.log_softmax(estudante_logits / T, dim=1)

    suave_loss = F.kl_div(estudante_log_probs, professor_probs, reduction="batchmean") * (T **2)

    entropia_cruzada_loss = F.cross_entropy(estudante_logits, targets)

    return alpha * entropia_cruzada_loss + (1 - alpha) * suave_loss


## Treinamento com destilação

In [253]:
otimizador  = torch.optim.Adam(aluno.parameters(), lr=0.01)

### Simulando congelamento de pesos do modelo

In [158]:
#congela o professor  (simula que já foi treinado)
# for param in professor.parameters():
#     param.requires_grad = False

### Simulando treinamento do modelo

In [254]:
#pretreinamento de professor
for batch in train_loader:
    x, y = batch
    logits = professor(x)
    loss = F.cross_entropy(logits,y)
    loss.backward()

    for param in professor.parameters():
        param.data -= 0.01 * param.grad

    professor.zero_grad()

In [255]:
#Treinamentode aluno com destilação
for epoca in range(10):
    total_loss = 0
    for x,y in train_loader:
        with torch.no_grad():
            professor_logits = professor(x)
        estudante_logits = aluno(x)
        loss = destilacao_loss(estudante_logits, professor_logits, y)
        loss.backward()
        otimizador.step()
        total_loss += loss.item()
    print(f"Epoca {epoca + 1}, Loss: {total_loss:.4f}")

Epoca 1, Loss: 9.5997
Epoca 2, Loss: 6.6988
Epoca 3, Loss: 7.0081
Epoca 4, Loss: 10.2467
Epoca 5, Loss: 20.9168
Epoca 6, Loss: 11.6950
Epoca 7, Loss: 16.4360
Epoca 8, Loss: 14.6982
Epoca 9, Loss: 11.7947
Epoca 10, Loss: 11.7744


### Avaliação   


In [256]:
def avaliacao(model, DataLoader):
    correto, total = 0 , 0
    model.eval()
    with torch.no_grad():
        for x, y in DataLoader:
            pred = model(x).argmax(dim=1)
            correto += (pred == y).sum().item()
            total += y.size(0)
    return correto / total



In [257]:
acc = avaliacao(aluno, test_loader)
print("acuracaia do  aluno: ", acc)

acuracaia do  aluno:  0.2


# Teste 3

Utilizando BERT e realizando fine tune sem destilação de conhecimento

In [12]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [13]:
# 1. Leitura do dataset
df = pd.read_csv("/content/textos_categorizados.csv").drop_duplicates(subset="texto")

In [14]:
label_enc = LabelEncoder()
df["label_encoded"] = label_enc.fit_transform(df["label"])

In [15]:

x_train, x_test, y_train, y_test = train_test_split(
    df["texto"], df["label_encoded"], test_size=0.2, random_state=42
)

In [16]:
# 4. Tokenização BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

class BERTDataset(Dataset):
    def __init__(self, textos, labels, tokenizer, max_len=64):
        self.encodings = tokenizer(
            list(textos),
            truncation=True,
            padding=True,
            max_length=max_len,
            return_tensors="pt"
        )
        self.labels = torch.tensor(labels.values)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

In [22]:
# 5. Criar Datasets e Dataloaders
train_dataset = BERTDataset(x_train, y_train, tokenizer)
test_dataset = BERTDataset(x_test, y_test, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

In [23]:
# 6. Modelo com BERT
class BERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = self.dropout(output.pooler_output)
        return self.fc(x)

In [24]:
# 7. Inicializar modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(num_classes=len(label_enc.classes_)).to(device)

In [25]:
# 8. Otimizador e função de perda
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

In [26]:
# 9. Loop de treino básico
for epoch in range(60):  # número de épocas
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")



Epoch 1, Loss: 2.7330
Epoch 2, Loss: 2.9451
Epoch 3, Loss: 2.3037
Epoch 4, Loss: 2.5967
Epoch 5, Loss: 2.2182
Epoch 6, Loss: 2.2155
Epoch 7, Loss: 2.1585
Epoch 8, Loss: 2.1273
Epoch 9, Loss: 1.8921
Epoch 10, Loss: 1.8634
Epoch 11, Loss: 1.9103
Epoch 12, Loss: 1.6741
Epoch 13, Loss: 1.7145
Epoch 14, Loss: 1.6682
Epoch 15, Loss: 1.4653
Epoch 16, Loss: 1.5160
Epoch 17, Loss: 1.3283
Epoch 18, Loss: 1.1798
Epoch 19, Loss: 1.0975
Epoch 20, Loss: 1.0623
Epoch 21, Loss: 0.9291
Epoch 22, Loss: 0.9215
Epoch 23, Loss: 0.9557
Epoch 24, Loss: 0.7785
Epoch 25, Loss: 0.7376
Epoch 26, Loss: 0.6233
Epoch 27, Loss: 0.6048
Epoch 28, Loss: 0.6430
Epoch 29, Loss: 0.5790
Epoch 30, Loss: 0.4451
Epoch 31, Loss: 0.4206
Epoch 32, Loss: 0.4147
Epoch 33, Loss: 0.3896
Epoch 34, Loss: 0.2897
Epoch 35, Loss: 0.3967
Epoch 36, Loss: 0.3025
Epoch 37, Loss: 0.2831
Epoch 38, Loss: 0.2759
Epoch 39, Loss: 0.2863
Epoch 40, Loss: 0.1885
Epoch 41, Loss: 0.2002
Epoch 42, Loss: 0.1977
Epoch 43, Loss: 0.1936
Epoch 44, Loss: 0.19

In [33]:
model.eval()
predicoes = []
textos_originais = list(x_test.reset_index(drop=True))  # para recuperar os textos

with torch.no_grad():
    for i, batch in enumerate(test_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probs = torch.softmax(outputs, dim=1)
        preds = torch.argmax(probs, dim=1)

        for j in range(len(labels)):
            texto = textos_originais[i * test_loader.batch_size + j]
            real_label = label_enc.inverse_transform([labels[j].cpu().item()])[0]
            pred_label = label_enc.inverse_transform([preds[j].cpu().item()])[0]

            print(f"\n📝 Texto: {texto}")
            print(f"✅ Real: {real_label} | 🔮 Previsto: {pred_label}")



📝 Texto: Organizações civis e movimentos sociais convocaram um ato nacional em resposta à violência policial registrada nos últimos meses nas periferias urbanas.
✅ Real: protesto | 🔮 Previsto: educacao

📝 Texto: O aumento dos focos de queimadas na região amazônica preocupa autoridades ambientais, que alertam para o risco de desequilíbrio ecológico e perda de biodiversidade.
✅ Real: meio_ambiente | 🔮 Previsto: meio_ambiente

📝 Texto: Estudos mostram que a poluição dos rios urbanos continua crescendo, mesmo com políticas de saneamento, o que representa riscos à saúde pública e à biodiversidade aquática.
✅ Real: meio_ambiente | 🔮 Previsto: meio_ambiente

📝 Texto: Milhares de manifestantes ocuparam as ruas da capital nesta tarde para protestar contra a reforma trabalhista aprovada pelo congresso nacional no mês passado.
✅ Real: protesto | 🔮 Previsto: economia


#TESTE 4

Destilação de conhecimento com BERT

In [46]:
import torch.nn.functional as F
from transformers import DistilBertTokenizer, DistilBertModel
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch

In [47]:
from sklearn.metrics import accuracy_score

In [48]:
from transformers import BertTokenizer, BertModel

In [49]:
class BERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = self.dropout(output.pooler_output)
        return self.fc(x)

In [50]:
# Define o modelo aluno (menor)
class DistilBERTStudent(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = self.dropout(output.last_hidden_state[:, 0])  # CLS token
        return self.fc(x)

In [51]:
df = pd.read_csv("/content/textos_categorizados.csv").drop_duplicates(subset="texto").reset_index(drop=True)

In [52]:
df

,texto,label
0,Organizações civis e movimentos sociais convoc...,protesto
1,Milhares de manifestantes ocuparam as ruas da ...,protesto
2,Protestos se intensificaram após o aumento no ...,protesto
3,Integrantes de diversos sindicatos realizaram ...,protesto
4,Estudantes de várias universidades públicas or...,protesto
5,O relatório do Banco Central apontou crescimen...,economia
6,A inflação do mês de março apresentou uma leve...,economia
7,Analistas financeiros revisaram as projeções d...,economia
8,A taxa básica de juros foi reduzida pelo Comit...,economia
9,A bolsa de valores fechou o pregão com alta de...,economia


In [53]:
num_classes = df["label"].nunique()
num_classes

4

In [54]:
label_enc = LabelEncoder()
df["label_encoded"] = label_enc.fit_transform(df["label"])

In [55]:
x_train, x_test, y_train, y_test = train_test_split(
    df["texto"], df["label_encoded"], test_size=0.2, random_state=42
)

In [56]:
# Tokenização BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

class BERTDataset(Dataset):
    def __init__(self, textos, labels, tokenizer, max_len=64):
        self.encodings = tokenizer(
            list(textos),
            truncation=True,
            padding=True,
            max_length=max_len,
            return_tensors="pt"
        )
        self.labels = torch.tensor(labels.values)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

In [57]:
# 5. Criar Datasets e Dataloaders
train_dataset = BERTDataset(x_train, y_train, tokenizer)
test_dataset = BERTDataset(x_test, y_test, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

In [58]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [59]:
model = BERTClassifier(num_classes=len(label_enc.classes_)).to(device)

In [60]:
# Instanciar modelos
professor_model = BERTClassifier(num_classes).to(device)
student_model = DistilBERTStudent(num_classes).to(device)

In [61]:
# 8. Otimizador e função de perda do professor
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

In [34]:
# Congela o professor (como se já estivesse treinado)
# for param in professor_model.parameters():
#     param.requires_grad = False

In [63]:
# 9. Loop de treino básico
for epoch in range(40):  # número de épocas
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 2.7765
Epoch 2, Loss: 2.7777
Epoch 3, Loss: 2.5519
Epoch 4, Loss: 2.4598
Epoch 5, Loss: 2.5610
Epoch 6, Loss: 2.4650
Epoch 7, Loss: 2.1153
Epoch 8, Loss: 2.0771
Epoch 9, Loss: 2.0133
Epoch 10, Loss: 1.9233
Epoch 11, Loss: 1.8337
Epoch 12, Loss: 1.8010
Epoch 13, Loss: 1.7112
Epoch 14, Loss: 1.7038
Epoch 15, Loss: 1.4637
Epoch 16, Loss: 1.4066
Epoch 17, Loss: 1.4782
Epoch 18, Loss: 1.3947
Epoch 19, Loss: 1.3554
Epoch 20, Loss: 1.1320
Epoch 21, Loss: 1.0182
Epoch 22, Loss: 0.8958
Epoch 23, Loss: 0.9520
Epoch 24, Loss: 0.7648
Epoch 25, Loss: 0.6881
Epoch 26, Loss: 0.6331
Epoch 27, Loss: 0.7072
Epoch 28, Loss: 0.6520
Epoch 29, Loss: 0.6104
Epoch 30, Loss: 0.5441
Epoch 31, Loss: 0.5093
Epoch 32, Loss: 0.4453
Epoch 33, Loss: 0.4778
Epoch 34, Loss: 0.4297
Epoch 35, Loss: 0.4296
Epoch 36, Loss: 0.4265
Epoch 37, Loss: 0.3926
Epoch 38, Loss: 0.3444
Epoch 39, Loss: 0.3504
Epoch 40, Loss: 0.3432


In [64]:
# Otimizador do aluno
optimizer = torch.optim.Adam(student_model.parameters(), lr=1e-5)

In [65]:
# Função de perda com destilação
def distilacao_loss(student_logits, teacher_logits, true_labels, T=2.0, alpha=0.7):
    # Soft targets do professor
    soft_teacher_probs = F.softmax(teacher_logits / T, dim=1)
    # Log softmax do aluno
    log_student_probs = F.log_softmax(student_logits / T, dim=1)
    # KL-divergence
    soft_loss = F.kl_div(log_student_probs, soft_teacher_probs, reduction="batchmean") * (T ** 2)
    # Loss tradicional
    hard_loss = F.cross_entropy(student_logits, true_labels)
    return alpha * hard_loss + (1 - alpha) * soft_loss

In [71]:
# Treinamento com destilação
for epoca in range(40):
    student_model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            teacher_logits = professor_model(input_ids=input_ids, attention_mask=attention_mask)

        student_logits = student_model(input_ids=input_ids, attention_mask=attention_mask)
        loss = distilacao_loss(student_logits, teacher_logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"[Epoca {epoca+1}] Distilacao Loss: {total_loss:.4f}")


[Epoca 1] Distilacao Loss: 1.0424
[Epoca 2] Distilacao Loss: 1.0692
[Epoca 3] Distilacao Loss: 1.0066
[Epoca 4] Distilacao Loss: 0.9747
[Epoca 5] Distilacao Loss: 0.9278
[Epoca 6] Distilacao Loss: 0.8351
[Epoca 7] Distilacao Loss: 0.8531
[Epoca 8] Distilacao Loss: 0.7968
[Epoca 9] Distilacao Loss: 0.8298
[Epoca 10] Distilacao Loss: 0.7908
[Epoca 11] Distilacao Loss: 0.8201
[Epoca 12] Distilacao Loss: 0.7517
[Epoca 13] Distilacao Loss: 0.8052
[Epoca 14] Distilacao Loss: 0.7946
[Epoca 15] Distilacao Loss: 0.7444
[Epoca 16] Distilacao Loss: 0.7748
[Epoca 17] Distilacao Loss: 0.7459
[Epoca 18] Distilacao Loss: 0.7530
[Epoca 19] Distilacao Loss: 0.7623
[Epoca 20] Distilacao Loss: 0.7931
[Epoca 21] Distilacao Loss: 0.8016
[Epoca 22] Distilacao Loss: 0.7686
[Epoca 23] Distilacao Loss: 0.7762
[Epoca 24] Distilacao Loss: 0.7564
[Epoca 25] Distilacao Loss: 0.7459
[Epoca 26] Distilacao Loss: 0.7505
[Epoca 27] Distilacao Loss: 0.7387
[Epoca 28] Distilacao Loss: 0.7915
[Epoca 29] Distilacao Loss: 0

In [72]:
student_model.eval()
all_preds = []
all_labels = []
textos_originais = list(x_test.reset_index(drop=True))

In [73]:
with torch.no_grad():
    for i, batch in enumerate(test_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = student_model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(F.softmax(outputs, dim=1), dim=1)

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

        for j in range(len(labels)):
            texto = textos_originais[i * test_loader.batch_size + j]
            real_label = label_enc.inverse_transform([labels[j].cpu().item()])[0]
            pred_label = label_enc.inverse_transform([preds[j].cpu().item()])[0]

            print(f"\n📝 Texto: {texto}")
            print(f"✅ Real: {real_label} | 🔮 Previsto: {pred_label}")


📝 Texto: Organizações civis e movimentos sociais convocaram um ato nacional em resposta à violência policial registrada nos últimos meses nas periferias urbanas.
✅ Real: protesto | 🔮 Previsto: meio_ambiente

📝 Texto: O aumento dos focos de queimadas na região amazônica preocupa autoridades ambientais, que alertam para o risco de desequilíbrio ecológico e perda de biodiversidade.
✅ Real: meio_ambiente | 🔮 Previsto: economia

📝 Texto: Estudos mostram que a poluição dos rios urbanos continua crescendo, mesmo com políticas de saneamento, o que representa riscos à saúde pública e à biodiversidade aquática.
✅ Real: meio_ambiente | 🔮 Previsto: economia

📝 Texto: Milhares de manifestantes ocuparam as ruas da capital nesta tarde para protestar contra a reforma trabalhista aprovada pelo congresso nacional no mês passado.
✅ Real: protesto | 🔮 Previsto: economia


In [34]:
# Calcular acurácia
acc = accuracy_score(all_labels, all_preds)
print(f"\n🎯 Acurácia final do aluno: {acc:.2%}")


🎯 Acurácia final do aluno: 25.00%


## Salvando o modelo

In [40]:
# Salvar os modelos
torch.save(student_model.state_dict(), "aluno_distilbert.pth")
torch.save(professor_model.state_dict(), "professor_bert.pth")


## Carrega modelo

In [ ]:
student_model.load_state_dict(torch.load("aluno_distilbert.pth"))
student_model.eval()


In [ ]:
all_preds = []
all_labels = []
textos_originais = list(x_test.reset_index(drop=True))

In [ ]:
with torch.no_grad():
    for i, batch in enumerate(test_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = student_model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(F.softmax(outputs, dim=1), dim=1)

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

        for j in range(len(labels)):
            texto = textos_originais[i * test_loader.batch_size + j]
            real_label = label_enc.inverse_transform([labels[j].cpu().item()])[0]
            pred_label = label_enc.inverse_transform([preds[j].cpu().item()])[0]

            print(f"\n📝 Texto: {texto}")
            print(f"✅ Real: {real_label} | 🔮 Previsto: {pred_label}")

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, DistilBertTokenizer, DistilBertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pandas as pd

In [7]:
# ======================= CONFIG ======================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 8
EPOCHS_PROF = 50
EPOCHS_ALUNO = 50
MAX_LEN = 64
ALPHA = 0.7
TEMPERATURA = 2.0

# ======================= DATA ========================
df = pd.read_csv("/content/textos_categorizados.csv").drop_duplicates(subset="texto")
label_enc = LabelEncoder()
df["label_encoded"] = label_enc.fit_transform(df["label"])
x_train, x_test, y_train, y_test = train_test_split(df["texto"], df["label_encoded"], test_size=0.2, random_state=42)

# Tokenização BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

class BERTDataset(Dataset):
    def __init__(self, textos, labels, tokenizer, max_len):
        self.encodings = tokenizer(list(textos), truncation=True, padding=True, max_length=max_len, return_tensors="pt")
        self.labels = torch.tensor(labels.values)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

train_dataset = BERTDataset(x_train, y_train, tokenizer, MAX_LEN)
test_dataset = BERTDataset(x_test, y_test, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)



In [8]:
# ======================= MODELOS ========================
class BERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = self.dropout(out.pooler_output)
        return self.fc(x)

class DistilBERTStudent(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = self.dropout(out.last_hidden_state[:, 0])
        return self.fc(x)

In [9]:
# ======================= TREINAR PROFESSOR ========================
professor = BERTClassifier(num_classes=len(label_enc.classes_)).to(device)
optimizer = torch.optim.AdamW(professor.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

print("\n🔧 Treinando professor...")
for epoch in range(EPOCHS_PROF):
    professor.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = professor(input_ids=input_ids, attention_mask=mask)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"📘 Epoca {epoch+1} - Loss: {total_loss:.4f}")


🔧 Treinando professor...
📘 Epoca 1 - Loss: 2.7574
📘 Epoca 2 - Loss: 2.9001
📘 Epoca 3 - Loss: 2.6948
📘 Epoca 4 - Loss: 2.8704
📘 Epoca 5 - Loss: 2.6725
📘 Epoca 6 - Loss: 2.7777
📘 Epoca 7 - Loss: 2.8965
📘 Epoca 8 - Loss: 2.7009
📘 Epoca 9 - Loss: 2.6861
📘 Epoca 10 - Loss: 2.7364
📘 Epoca 11 - Loss: 2.7349
📘 Epoca 12 - Loss: 2.6819
📘 Epoca 13 - Loss: 2.4146
📘 Epoca 14 - Loss: 2.3657
📘 Epoca 15 - Loss: 2.2269
📘 Epoca 16 - Loss: 2.1452
📘 Epoca 17 - Loss: 1.9721
📘 Epoca 18 - Loss: 2.0251
📘 Epoca 19 - Loss: 1.5475
📘 Epoca 20 - Loss: 1.6175
📘 Epoca 21 - Loss: 1.5031
📘 Epoca 22 - Loss: 1.4128
📘 Epoca 23 - Loss: 1.1822
📘 Epoca 24 - Loss: 1.1494
📘 Epoca 25 - Loss: 0.9942
📘 Epoca 26 - Loss: 0.9995
📘 Epoca 27 - Loss: 0.8305
📘 Epoca 28 - Loss: 0.8068
📘 Epoca 29 - Loss: 0.6562
📘 Epoca 30 - Loss: 0.5472
📘 Epoca 31 - Loss: 0.5273
📘 Epoca 32 - Loss: 0.4512
📘 Epoca 33 - Loss: 0.4916
📘 Epoca 34 - Loss: 0.3441
📘 Epoca 35 - Loss: 0.2985
📘 Epoca 36 - Loss: 0.3447
📘 Epoca 37 - Loss: 0.2637
📘 Epoca 38 - Loss: 0.

In [10]:
# ======================= TREINAR ALUNO COM DESTILAÇÃO ========================
aluno = DistilBERTStudent(num_classes=len(label_enc.classes_)).to(device)
optimizer_aluno = torch.optim.AdamW(aluno.parameters(), lr=3e-5)

def distillation_loss(student_logits, teacher_logits, true_labels, T=2.0, alpha=0.7):
    soft_teacher = F.softmax(teacher_logits / T, dim=1)
    log_student = F.log_softmax(student_logits / T, dim=1)
    loss_soft = F.kl_div(log_student, soft_teacher, reduction="batchmean") * (T ** 2)
    loss_hard = F.cross_entropy(student_logits, true_labels)
    return alpha * loss_hard + (1 - alpha) * loss_soft

print("\n🎓 Treinando aluno com distilação...")
professor.eval()  # Desliga gradientes do professor
for epoca in range(EPOCHS_ALUNO):
    aluno.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            teacher_logits = professor(input_ids=input_ids, attention_mask=mask)

        student_logits = aluno(input_ids=input_ids, attention_mask=mask)
        loss = distillation_loss(student_logits, teacher_logits, labels, T=TEMPERATURA, alpha=ALPHA)

        optimizer_aluno.zero_grad()
        loss.backward()
        optimizer_aluno.step()
        total_loss += loss.item()
    print(f"🎯 Epoca {epoca+1} - Distilação Loss: {total_loss:.4f}")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]


🎓 Treinando aluno com distilação...
🎯 Epoch 1 - Distillation Loss: 3.8061
🎯 Epoch 2 - Distillation Loss: 3.2270
🎯 Epoch 3 - Distillation Loss: 3.1382
🎯 Epoch 4 - Distillation Loss: 3.0860
🎯 Epoch 5 - Distillation Loss: 3.0247
🎯 Epoch 6 - Distillation Loss: 3.0320
🎯 Epoch 7 - Distillation Loss: 2.8187
🎯 Epoch 8 - Distillation Loss: 2.6221
🎯 Epoch 9 - Distillation Loss: 2.4761
🎯 Epoch 10 - Distillation Loss: 2.1060
🎯 Epoch 11 - Distillation Loss: 1.5790
🎯 Epoch 12 - Distillation Loss: 1.4014
🎯 Epoch 13 - Distillation Loss: 1.2687
🎯 Epoch 14 - Distillation Loss: 0.8073
🎯 Epoch 15 - Distillation Loss: 0.5931
🎯 Epoch 16 - Distillation Loss: 0.4758
🎯 Epoch 17 - Distillation Loss: 0.3099
🎯 Epoch 18 - Distillation Loss: 0.2001
🎯 Epoch 19 - Distillation Loss: 0.1873
🎯 Epoch 20 - Distillation Loss: 0.0993
🎯 Epoch 21 - Distillation Loss: 0.0832
🎯 Epoch 22 - Distillation Loss: 0.0637
🎯 Epoch 23 - Distillation Loss: 0.0633
🎯 Epoch 24 - Distillation Loss: 0.0695
🎯 Epoch 25 - Distillation Loss: 0.08

In [11]:
# ======================= AVALIAÇÃO ========================
aluno.eval()
all_preds, all_labels = [], []
textos_originais = list(x_test.reset_index(drop=True))

with torch.no_grad():
    for i, batch in enumerate(test_loader):
        input_ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = aluno(input_ids=input_ids, attention_mask=mask)
        preds = torch.argmax(F.softmax(outputs, dim=1), dim=1)

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

        for j in range(len(labels)):
            texto = textos_originais[i * BATCH_SIZE + j]
            real = label_enc.inverse_transform([labels[j].cpu().item()])[0]
            pred = label_enc.inverse_transform([preds[j].cpu().item()])[0]
            print(f"\n📝 Texto: {texto}")
            print(f"✅ Real: {real} | 🔮 Previsto: {pred}")

acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Acurácia final do aluno: {acc:.2%}")



📝 Texto: Organizações civis e movimentos sociais convocaram um ato nacional em resposta à violência policial registrada nos últimos meses nas periferias urbanas.
✅ Real: protesto | 🔮 Previsto: meio_ambiente

📝 Texto: O aumento dos focos de queimadas na região amazônica preocupa autoridades ambientais, que alertam para o risco de desequilíbrio ecológico e perda de biodiversidade.
✅ Real: meio_ambiente | 🔮 Previsto: meio_ambiente

📝 Texto: Estudos mostram que a poluição dos rios urbanos continua crescendo, mesmo com políticas de saneamento, o que representa riscos à saúde pública e à biodiversidade aquática.
✅ Real: meio_ambiente | 🔮 Previsto: protesto

📝 Texto: Milhares de manifestantes ocuparam as ruas da capital nesta tarde para protestar contra a reforma trabalhista aprovada pelo congresso nacional no mês passado.
✅ Real: protesto | 🔮 Previsto: economia

✅ Acurácia final do aluno: 25.00%


In [35]:
# ======================= AVALIAÇÃO DO PROFESSOR ========================
professor.eval()
all_preds_professor, all_labels_professor = [], []

print("\n🧑‍🏫 Avaliação do professor:")

with torch.no_grad():
    for i, batch in enumerate(test_loader):
        input_ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = professor(input_ids=input_ids, attention_mask=mask)
        preds = torch.argmax(F.softmax(outputs, dim=1), dim=1)

        all_preds_professor.extend(preds.cpu().tolist())
        all_labels_professor.extend(labels.cpu().tolist())

        for j in range(len(labels)):
            texto = textos_originais[i * BATCH_SIZE + j]
            real = label_enc.inverse_transform([labels[j].cpu().item()])[0]
            pred = label_enc.inverse_transform([preds[j].cpu().item()])[0]
            print(f"\n📘 Texto: {texto}")
            print(f"✅ Real: {real} | 🧑‍🏫 Previsto (Professor): {pred}")

acc_prof = accuracy_score(all_labels_professor, all_preds_professor)
print(f"\n🎯 Acurácia final do professor: {acc_prof:.2%}")



🧑‍🏫 Avaliação do professor:

📘 Texto: Organizações civis e movimentos sociais convocaram um ato nacional em resposta à violência policial registrada nos últimos meses nas periferias urbanas.
✅ Real: protesto | 🧑‍🏫 Previsto (Professor): educacao

📘 Texto: O aumento dos focos de queimadas na região amazônica preocupa autoridades ambientais, que alertam para o risco de desequilíbrio ecológico e perda de biodiversidade.
✅ Real: meio_ambiente | 🧑‍🏫 Previsto (Professor): meio_ambiente

📘 Texto: Estudos mostram que a poluição dos rios urbanos continua crescendo, mesmo com políticas de saneamento, o que representa riscos à saúde pública e à biodiversidade aquática.
✅ Real: meio_ambiente | 🧑‍🏫 Previsto (Professor): meio_ambiente

📘 Texto: Milhares de manifestantes ocuparam as ruas da capital nesta tarde para protestar contra a reforma trabalhista aprovada pelo congresso nacional no mês passado.
✅ Real: protesto | 🧑‍🏫 Previsto (Professor): protesto

🎯 Acurácia final do professor: 75.00%


## Realizando aprendizado por extração de feature e comparando com Fine tune

In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pandas as pd

In [55]:
# ============ CONFIG =============
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LEN = 64
BATCH_SIZE = 8
EPOCHS = 50

In [56]:
df = pd.read_csv("/content/textos_categorizados.csv").drop_duplicates(subset="texto")
df

,texto,label
0,Organizações civis e movimentos sociais convoc...,protesto
1,Milhares de manifestantes ocuparam as ruas da ...,protesto
4,Protestos se intensificaram após o aumento no ...,protesto
10,Integrantes de diversos sindicatos realizaram ...,protesto
13,Estudantes de várias universidades públicas or...,protesto
25,O relatório do Banco Central apontou crescimen...,economia
27,A inflação do mês de março apresentou uma leve...,economia
28,Analistas financeiros revisaram as projeções d...,economia
29,A taxa básica de juros foi reduzida pelo Comit...,economia
35,A bolsa de valores fechou o pregão com alta de...,economia


In [57]:
# ============ DATA ==============

label_enc = LabelEncoder()
df["label_encoded"] = label_enc.fit_transform(df["label"])

In [58]:
x_train, x_test, y_train, y_test = train_test_split(df["texto"], df["label_encoded"], test_size=0.2, random_state=42)

In [59]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [60]:
class BERTDataset(Dataset):
    def __init__(self, textos, labels, tokenizer, max_len):
        self.encodings = tokenizer(list(textos), truncation=True, padding=True, max_length=max_len, return_tensors="pt")
        self.labels = torch.tensor(labels.values)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

In [61]:
train_dataset = BERTDataset(x_train, y_train, tokenizer, MAX_LEN)
test_dataset = BERTDataset(x_test, y_test, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [62]:
# ============ MODELOS ==============
class BERTFineTuned(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = self.dropout(outputs.pooler_output)
        return self.fc(x)



In [63]:
class BERTFeatureExtractor(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        for param in self.bert.parameters():
            param.requires_grad = False  # congela BERT
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = self.dropout(outputs.pooler_output)
        return self.fc(x)

In [64]:
# ============ TREINAMENTO ===========
def treinar_modelo(model, dataloader, epochs=3):
    model.to(device)
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)
    criterion = nn.CrossEntropyLoss()

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=mask)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"📚 Epoch {epoch+1}/{epochs} | Loss: {total_loss:.4f}")
    return model

In [65]:
# ============ AVALIAÇÃO ===========
def avaliar_modelo(model, dataloader, textos_originais, name=""):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            input_ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=mask)
            preds = torch.argmax(F.softmax(outputs, dim=1), dim=1)

            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())

            for j in range(len(labels)):
                texto = textos_originais[i * BATCH_SIZE + j]
                real = label_enc.inverse_transform([labels[j].cpu().item()])[0]
                pred = label_enc.inverse_transform([preds[j].cpu().item()])[0]
                print(f"\n🔎 [{name}] Texto: {texto[:100]}...")
                print(f"✅ Real: {real} | 🤖 Previsto: {pred}")

    acc = accuracy_score(all_labels, all_preds)
    print(f"\n✅ Acurácia final do {name}: {acc:.2%}")
    return acc

In [66]:
# ============ EXECUÇÃO ============

print("\n🎯 Treinando modelo com Fine-Tuning completo:")
model_fine = BERTFineTuned(num_classes=len(label_enc.classes_))
model_fine = treinar_modelo(model_fine, train_loader, epochs=EPOCHS)

print("\n🎯 Treinando modelo com Feature Extraction:")
model_feat = BERTFeatureExtractor(num_classes=len(label_enc.classes_))
model_feat = treinar_modelo(model_feat, train_loader, epochs=EPOCHS)

# Avaliação lado a lado
print("\n📊 Avaliação comparativa:")
avaliar_modelo(model_fine, test_loader, list(x_test.reset_index(drop=True)), name="Fine-Tuning")
avaliar_modelo(model_feat, test_loader, list(x_test.reset_index(drop=True)), name="Feature Extraction")


🎯 Treinando modelo com Fine-Tuning completo:
📚 Epoch 1/50 | Loss: 2.9801
📚 Epoch 2/50 | Loss: 2.8921
📚 Epoch 3/50 | Loss: 2.8685
📚 Epoch 4/50 | Loss: 2.6018
📚 Epoch 5/50 | Loss: 2.5297
📚 Epoch 6/50 | Loss: 2.6343
📚 Epoch 7/50 | Loss: 2.6032
📚 Epoch 8/50 | Loss: 2.5449
📚 Epoch 9/50 | Loss: 2.4529
📚 Epoch 10/50 | Loss: 2.5284
📚 Epoch 11/50 | Loss: 2.6304
📚 Epoch 12/50 | Loss: 2.2764
📚 Epoch 13/50 | Loss: 2.1913
📚 Epoch 14/50 | Loss: 2.2917
📚 Epoch 15/50 | Loss: 1.9980
📚 Epoch 16/50 | Loss: 2.1789
📚 Epoch 17/50 | Loss: 1.8702
📚 Epoch 18/50 | Loss: 1.9771
📚 Epoch 19/50 | Loss: 1.7417
📚 Epoch 20/50 | Loss: 1.7473
📚 Epoch 21/50 | Loss: 1.7135
📚 Epoch 22/50 | Loss: 1.6422
📚 Epoch 23/50 | Loss: 1.6502
📚 Epoch 24/50 | Loss: 1.5184
📚 Epoch 25/50 | Loss: 1.4969
📚 Epoch 26/50 | Loss: 1.4285
📚 Epoch 27/50 | Loss: 1.3053
📚 Epoch 28/50 | Loss: 1.4932
📚 Epoch 29/50 | Loss: 1.3273
📚 Epoch 30/50 | Loss: 1.2586
📚 Epoch 31/50 | Loss: 1.1926
📚 Epoch 32/50 | Loss: 1.0214
📚 Epoch 33/50 | Loss: 1.1121
📚 Epoc

0.0

# Comparando aprendizado com modelo sendo treinado com Fine tuning, extração de feature e LoRa

Fine-tuning completo (tudo treinável)

Feature extraction (BERT congelado)

LoRA (adaptação leve)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from peft import get_peft_model, LoraConfig, TaskType
import pandas as pd

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LEN = 64
BATCH_SIZE = 8
EPOCHS = 30

In [69]:
# ========== LOAD DATA ==========
df = pd.read_csv("textos_categorizados.csv").drop_duplicates(subset="texto")

In [70]:
label_enc = LabelEncoder()
df["label_encoded"] = label_enc.fit_transform(df["label"])

In [71]:
x_train, x_test, y_train, y_test = train_test_split(df["texto"], df["label_encoded"], test_size=0.2, random_state=42)

In [73]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [74]:
class BERTDataset(Dataset):
    def __init__(self, textos, labels, tokenizer, max_len):
        self.encodings = tokenizer(list(textos), truncation=True, padding=True, max_length=max_len, return_tensors="pt")
        self.labels = torch.tensor(labels.values)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item



In [76]:
train_dataset = BERTDataset(x_train, y_train, tokenizer, MAX_LEN)
test_dataset = BERTDataset(x_test, y_test, tokenizer, MAX_LEN)


In [77]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [78]:
# ========== MODELS ==========
class BERTFineTuned(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = self.dropout(outputs.pooler_output)
        return self.fc(x)

In [79]:
class BERTFeatureExtractor(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        for param in self.bert.parameters():
            param.requires_grad = False
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = self.dropout(outputs.pooler_output)
        return self.fc(x)

In [84]:
def get_lora_model(num_classes):
    base_model = AutoModelForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=num_classes
    )
    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["query", "value"],
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.SEQ_CLS,
    )
    lora_model = get_peft_model(base_model, lora_config)
    return lora_model

In [85]:
# ========== Treinamento ==========
def treinar_modelo(model, dataloader, epochs=3):
    model.to(device)
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-5)
    criterion = nn.CrossEntropyLoss()
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=mask)
            loss = criterion(outputs, labels) if isinstance(outputs, torch.Tensor) else criterion(outputs.logits, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"📚 Epoch {epoch+1}/{epochs} | Loss: {total_loss:.4f}")
    return model

In [86]:
# ========== Avaliação ==========
def avaliar_modelo(model, dataloader, textos_originais, name=""):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            input_ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=mask)
            logits = outputs if isinstance(outputs, torch.Tensor) else outputs.logits
            preds = torch.argmax(F.softmax(logits, dim=1), dim=1)
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())
            for j in range(len(labels)):
                texto = textos_originais[i * BATCH_SIZE + j]
                real = label_enc.inverse_transform([labels[j].cpu().item()])[0]
                pred = label_enc.inverse_transform([preds[j].cpu().item()])[0]
                print(f"\n🔎 [{name}] Texto: {texto[:100]}...")
                print(f"✅ Real: {real} | 🤖 Previsto: {pred}")
    acc = accuracy_score(all_labels, all_preds)
    print(f"\n✅ Acurácia final do {name}: {acc:.2%}")
    return acc

In [87]:
# ========== Execução ==========
print("\n🎯 Treinando modelo com Fine-Tuning completo:")
model_fine = BERTFineTuned(num_classes=len(label_enc.classes_))
model_fine = treinar_modelo(model_fine, train_loader, epochs=EPOCHS)

print("\n🎯 Treinando modelo com Feature Extraction:")
model_feat = BERTFeatureExtractor(num_classes=len(label_enc.classes_))
model_feat = treinar_modelo(model_feat, train_loader, epochs=EPOCHS)

print("\n🎯 Treinando modelo com LoRA:")
model_lora = get_lora_model(num_classes=len(label_enc.classes_))
model_lora = treinar_modelo(model_lora, train_loader, epochs=EPOCHS)


🎯 Treinando modelo com Fine-Tuning completo:
📚 Epoch 1/3 | Loss: 2.7541
📚 Epoch 2/3 | Loss: 2.5195
📚 Epoch 3/3 | Loss: 2.6437

🎯 Treinando modelo com Feature Extraction:
📚 Epoch 1/3 | Loss: 2.9063
📚 Epoch 2/3 | Loss: 2.8264
📚 Epoch 3/3 | Loss: 3.1874

🎯 Treinando modelo com LoRA:


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


📚 Epoch 1/3 | Loss: 2.7983
📚 Epoch 2/3 | Loss: 2.8394
📚 Epoch 3/3 | Loss: 2.8715


In [88]:
# ========== Compração ==========
print("\n📊 Avaliação comparativa:")
avaliar_modelo(model_fine, test_loader, list(x_test.reset_index(drop=True)), name="Fine-Tuning")
avaliar_modelo(model_feat, test_loader, list(x_test.reset_index(drop=True)), name="Feature Extraction")
avaliar_modelo(model_lora, test_loader, list(x_test.reset_index(drop=True)), name="LoRA")


📊 Avaliação comparativa:

🔎 [Fine-Tuning] Texto: Organizações civis e movimentos sociais convocaram um ato nacional em resposta à violência policial ...
✅ Real: protesto | 🤖 Previsto: educacao

🔎 [Fine-Tuning] Texto: O aumento dos focos de queimadas na região amazônica preocupa autoridades ambientais, que alertam pa...
✅ Real: meio_ambiente | 🤖 Previsto: educacao

🔎 [Fine-Tuning] Texto: Estudos mostram que a poluição dos rios urbanos continua crescendo, mesmo com políticas de saneament...
✅ Real: meio_ambiente | 🤖 Previsto: educacao

🔎 [Fine-Tuning] Texto: Milhares de manifestantes ocuparam as ruas da capital nesta tarde para protestar contra a reforma tr...
✅ Real: protesto | 🤖 Previsto: educacao

✅ Acurácia final do Fine-Tuning: 0.00%

🔎 [Feature Extraction] Texto: Organizações civis e movimentos sociais convocaram um ato nacional em resposta à violência policial ...
✅ Real: protesto | 🤖 Previsto: meio_ambiente

🔎 [Feature Extraction] Texto: O aumento dos focos de queimadas na regiã

0.0